In [1]:
import sys
import threading

import grpc
import numpy
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from PIL import Image
import numpy as np
import cv2

In [11]:
def pretreat(inputs):
        
    crop_size = 160
    outputs = []
    for imggroup in inputs:
            
        group = []
        for image in imggroup:
                
            height = image.shape[0]
            width = image.shape[1]   
                        
            if(width>height):
                scale=float(crop_size)/float(height)
                img=np.array(cv2.resize(image,(int(width*scale+1),crop_size))).astype(np.float32)      
            else:                    
                scale=float(crop_size)/float(width)
                img=np.array(cv2.resize(image,(crop_size,int(height*scale+1)))).astype(np.float32)
                
            crop_y=int((img.shape[0]-crop_size)/2)
            crop_x=int((img.shape[1]-crop_size)/2)
            img=img[crop_y:crop_y+crop_size,crop_x:crop_x+crop_size,:]

            #print img
            
            std = np.std(img, ddof=1)
            mean = np.mean(img)
            std = np.max([std, 1.0/np.sqrt(160*160*3)])
           
            img = (img-mean)/std

            group.append(img)
            
        outputs.append(group)
    return np.array(outputs).astype(np.float32)

channel = grpc.insecure_channel('127.0.0.1:8500')
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

for _ in range(1):
    request = predict_pb2.PredictRequest()
    request.model_spec.name = '3daction'
    request.model_spec.signature_name = 'predict_image'
    
    dir_path = '/home/sw/Work/tf/yaochang/tests/'
    inputs = []
    for i in range(16):
        file_path = dir_path + '{:0>4}'.format(i) + '.jpg'
        img = Image.open(file_path)
        inputs.append(np.array(img))
        
    dir_path = '/home/sw/Work/tf/yaochang/test2/'
    inputs2 = []
    for i in range(16):
        file_path = dir_path + '{:0>4}'.format(i) + '.jpg'
        img = Image.open(file_path)
        inputs2.append(np.array(img))
        
    image = pretreat(np.array([inputs,inputs2]))
    
    request.inputs['x'].CopyFrom(
        tf.contrib.util.make_tensor_proto(image))
    
    result_future = stub.Predict.future(request, 10)  # 5 seconds
    exception = result_future.exception()
    if exception:
        print(exception)
    else:
        response = numpy.array(
        result_future.result().outputs['logit'].float_val)
        print response.reshape([2,-1])


[[38.65402985 39.78093719 41.80890274]
 [ 1.49255705  4.94189024 -3.93510246]]


In [10]:
def letterbox_image(image, size):
    '''resize image with unchanged aspect ratio using padding'''
    ih, iw, _ = image.shape
    w, h = size
    scale = min(w / iw, h / ih)
    nw = int(iw * scale)
    nh = int(ih * scale)

    image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)

    new_image = np.ones((416, 416, 3), dtype=np.int8) * 128
    new_image[(h - nh) // 2:(h - nh) // 2 + nh, (w - nw) // 2:(w - nw) // 2 + nw, :] = image

    return new_image


def pretreat(inputs):
    boxed_image = letterbox_image(inputs, tuple(reversed((416, 416))))
    image_data = np.array(boxed_image, dtype='float32')
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

    return image_data

channel = grpc.insecure_channel('127.0.0.1:8500')
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

request = predict_pb2.PredictRequest()
request.model_spec.name = 'yolov3'
request.model_spec.signature_name = 'detect_image'
    
file_path = '/home/sw/Work/tf/yaochang/test2/0000.jpg'

img = Image.open(file_path)
img = np.array(img)
image = pretreat(img)
    
request.inputs['x'].CopyFrom(tf.contrib.util.make_tensor_proto(image))
request.inputs['shape'].CopyFrom(tf.contrib.util.make_tensor_proto([float(img.shape[0]), float(img.shape[1])]))
    
result_future = stub.Predict.future(request, 10)  # 5 seconds
exception = result_future.exception()
if exception:
    print(exception)
else:
    response = numpy.array([
    result_future.result().outputs['boxes'].float_val,
    result_future.result().outputs['scores'].float_val,
    result_future.result().outputs['classes'].int_val])
    print response


[[2.3538780212402344, 28.962987899780273, 182.40249633789062, 127.52815246582031, 138.71530151367188, 45.7708854675293, 178.7674560546875, 107.84136199951172]
 [0.6710521578788757, 0.42240986227989197] [0, 0]]


In [ ]:
[[ 1.43970215e+02  4.11068993e+01  1.87833420e+02  1.12543106e+02]
 [-2.93624997e-02  5.80841370e+01  1.84964127e+02  1.54046722e+02]
 [ 6.47915125e-01  1.89538784e+01  1.84801239e+02  1.09820030e+02]] [0.603676   0.48700953 0.36339325] [0 0 0]